<a href="https://colab.research.google.com/github/danielhou13/cogs402longformer/blob/main/src/Token_attention_with_head_importance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A key feature of Transformer neural networks is the attention feature. This attention is generally a sequence length x sequence length matrix output for every layer, batch and head of an input. As such, for every layer, batch, head, we can find out information about each token, whether its about what tokens a particular token attends to, or the most attended to token for each matrix.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import sys
# sys.path.append('/content/drive/My Drive/{}'.format("cogs402longformer/"))

In [3]:
pip install datasets --quiet

     |████████████████████████████████| 362 kB 6.8 MB/s 
     |████████████████████████████████| 140 kB 69.0 MB/s 
     |████████████████████████████████| 101 kB 13.5 MB/s 
     |████████████████████████████████| 1.1 MB 75.2 MB/s 
     |████████████████████████████████| 212 kB 91.0 MB/s 
     |████████████████████████████████| 596 kB 57.6 MB/s 
     |████████████████████████████████| 127 kB 82.6 MB/s 
     |████████████████████████████████| 144 kB 87.2 MB/s 
     |████████████████████████████████| 94 kB 4.1 MB/s 
     |████████████████████████████████| 271 kB 89.4 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.1 MB/s 
     |████████████████████████████████| 6.6 MB 51.9 MB/s 


###Import Dataset and Model

In [5]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

Import the Reserach Papers dataset

In [6]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('allenai/longformer-base-4096')

def longformer_finetuned_papers():
    model = AutoModelForSequenceClassification.from_pretrained('danielhou13/longformer-finetuned_papers_v2', num_labels = 2)
    return model

def preprocess_function(tokenizer, example, max_length):
    example.update(tokenizer(example['text'], padding='max_length', max_length=max_length, truncation=True))
    return example

def get_papers_dataset(dataset_type):
    max_length = 2048
    dataset = load_dataset("danielhou13/cogs402dataset")[dataset_type]

    # tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    dataset = dataset.map(lambda x: preprocess_function(tokenizer, x, max_length), batched=True)
    setattr(dataset, 'input_columns', ['input_ids', 'attention_mask'])
    setattr(dataset, 'target_columns', ['labels'])
    setattr(dataset, 'max_length', max_length)
    setattr(dataset, 'tokenizer', tokenizer)
    return dataset

def papers_test_set():
    return get_papers_dataset('test')

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Import the news dataset

In [7]:
# def preprocess_function(tokenizer, example, max_length):
#     example.update(tokenizer(example['text'], padding='max_length', max_length=max_length, truncation=True))
#     return example

# def longformer_finetuned_news():
#     model = AutoModelForSequenceClassification.from_pretrained('danielhou13/longformer-finetuned-news-cogs402', num_labels = 2)
#     return model

# def get_news_dataset(dataset_type):
#     max_length = 2048
#     dataset = load_dataset("danielhou13/cogs402dataset2")[dataset_type]

#     tokenizer = AutoTokenizer.from_pretrained('allenai/longformer-base-4096')
#     dataset = dataset.map(lambda x: preprocess_function(tokenizer, x, max_length), batched=True)

#     labels = map(int, dataset['hyperpartisan'])
#     print(type(dataset['hyperpartisan']))
#     labels = list(labels)
#     dataset = dataset.add_column("labels", labels)

#     dataset = dataset.remove_columns(['title', 'hyperpartisan', 'url', 'published_at', 'bias'])
#     print(dataset)
#     setattr(dataset, 'input_columns', ['input_ids', 'attention_mask'])
#     setattr(dataset, 'target_columns', ['labels'])
#     setattr(dataset, 'max_length', max_length)
#     setattr(dataset, 'tokenizer', tokenizer)
#     return dataset

# def news_train_set():
#     return get_news_dataset('train')

# def news_test_set():
#     return get_news_dataset('validation')

Load papers model and dataset and preprocess it

In [8]:
cogs402_test = papers_test_set()
model = longformer_finetuned_papers()
columns = cogs402_test.input_columns + cogs402_test.target_columns
print(columns)
cogs402_test.set_format(type='torch', columns=columns)
cogs402_test=cogs402_test.remove_columns(['text'])

Downloading:   0%|          | 0.00/739 [00:00<?, ?B/s]

Using custom data configuration danielhou13--cogs402dataset-144b958ac1a53abb


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

...

Parameter 'function'=<function get_papers_dataset.<locals>.<lambda> at 0x7f4d41bc99e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/2 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/0.99k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/567M [00:00<?, ?B/s]

['input_ids', 'attention_mask', 'labels']


Load news model and dataset and preprocess it

In [9]:
# cogs402_test = news_test_set()
# model = longformer_finetuned_news()
# columns = cogs402_test.input_columns + cogs402_test.target_columns
# print(columns)
# cogs402_test.set_format(type='torch', columns=columns)

### Get attention output for example

In [10]:
if torch.cuda.is_available():
    model = model.cuda()

print(model.device)

cuda:0


In [11]:
test_val = [976]
print(test_val)
testexam = cogs402_test[test_val]

[976]


In [12]:
output = model(testexam["input_ids"].cuda(), attention_mask=testexam['attention_mask'].cuda(), labels=testexam['labels'].cuda(), output_attentions = True)
batch_attn = output[-2]
output_attentions = torch.stack(batch_attn).cpu()
global_attention = output[-1]
output_global_attentions = torch.stack(global_attention).cpu()
print("output_attention.shape", output_attentions.shape)
print("gl_output_attention.shape", output_global_attentions.shape)

output_attention.shape torch.Size([12, 1, 12, 2048, 514])
gl_output_attention.shape torch.Size([12, 1, 12, 2048, 1])


In [13]:
print(testexam['labels'][0])
print(output[1].argmax())

tensor(1)
tensor(1, device='cuda:0')


In [14]:
# print(os.getcwd())
# yes = torch.load("resources/longformer_test2/epoch_3/aggregate_attn.pt")

### Convert sliding window attention to traditional format

A unique property of the longformer modelis that the matrix output for the attention is not a seq_len x seq_len output. Each token can only attend to the preceeding x tokens and the succeeding x tokens, dictated by whatever you choose the model's attention window x to be. Another name for this is called the sliding window attendion. Therefore, we need to convert sliding attention matrix to correct seq_len x seq_len matrix to remain consistent with other types of Transformer Neural Networkss.

In [15]:
def create_head_matrix(output_attentions, global_attentions):
    new_attention_matrix = torch.zeros((output_attentions.shape[0], 
                                      output_attentions.shape[0]))
    for i in range(output_attentions.shape[0]):
        test_non_zeroes = torch.nonzero(output_attentions[i]).squeeze()
        test2 = output_attentions[i][test_non_zeroes[1:]]
        new_attention_matrix_indices = test_non_zeroes[1:]-257 + i
        new_attention_matrix[i][new_attention_matrix_indices] = test2
        new_attention_matrix[i][0] = output_attentions[i][0]
        new_attention_matrix[0] = global_attentions.squeeze()[:output_attentions.shape[0]]
    return new_attention_matrix


def attentions_all_heads(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = create_head_matrix(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

def all_batches(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = attentions_all_heads(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

def all_layers(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = all_batches(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

In [16]:
converted_mat = all_layers(output_attentions, output_global_attentions).detach().cpu().numpy()
print(converted_mat.shape)

(12, 1, 12, 2048, 2048)


In [17]:
print(testexam['input_ids'])

tensor([[    0, 29642,    25,  ...,  3156,     6,     2]])


Not all heads have the same impact on the final output. Load head importance model and scale the attentions by head importance so the important heads have more influence.

In [18]:
head_importance = torch.load("/content/drive/MyDrive/cogs402longformer/t3-visapplication/resources/papers/pretrained/head_importance.pt")
# head_importance = torch.load("/content/drive/MyDrive/cogs402longformer/t3-visapplication/resources/news/head_importance.pt")

In [19]:
def scale_by_importance(attention_matrix, head_importance):
  new_matrix = np.zeros_like(attention_matrix)
  for i in range(attention_matrix.shape[0]):
    head_importance_layer = head_importance[i]
    for j in range(attention_matrix.shape[1]):
      new_matrix[i,j] = attention_matrix[i,j] * np.expand_dims(head_importance_layer, axis=(1,2))
  return new_matrix

In [20]:
converted_mat_importance = scale_by_importance(converted_mat, head_importance)

### Assess attention Matrix

#### Every word's top attended tokens

Lets suppose we want the topk attended tokens for each token in each head, batch and layer. In other words, we want to know which tokens each token attends TO the most.

In [21]:
all_tokens = tokenizer.convert_ids_to_tokens(testexam["input_ids"][0])

In [22]:
# get the top k and indexes and values for each row
def find_top_attention_unsummed(scores_mat, k):
  indices = scores_mat.argsort(axis=4)[:, :, :, :, :-(k+1):-1]
  vals = np.take_along_axis(scores_mat, indices, axis=4)  
  return indices, vals

#find the tokens using the index matrix and the all_tokens list to create a 
#matrix of tokens 
def get_tokens(index_matrix, example=None):
  highest_tokens = []
  #layer
  for i in range(index_matrix.shape[0]):
    row_tokens = []
    #batch
    for j in range(index_matrix.shape[1]):
      batch_tokens = []

      if (example is not None) and (j != example):
        continue

      #head
      for k in range(index_matrix.shape[2]):
        head_tokens = []
        #token
        for x in range(index_matrix.shape[3]):
          tokens = [all_tokens[idx] for idx in index_matrix[i][j][k][x]]
          head_tokens.append(tokens)
        batch_tokens.append(head_tokens)
      row_tokens.append(batch_tokens)
    highest_tokens.append(row_tokens)
  return np.array(highest_tokens)

#format into a dataframe
def highest_attended_tokens(index, values, tokens, all_tokens, example=None):
    dataframe=[]
    for i in range(index.shape[0]):
      for j in range(index.shape[1]):

        if (example is not None) and (j != example):
          continue

        for k in range(index.shape[2]):
          for x in range(index.shape[3]):
            for y in range(index.shape[4]):
              d = {"token":all_tokens[x], 'self_position':x, 
                  "attended_token": tokens[i,j,k,x,y],
                  'token_position':index[i,j,k,x,y], 
                  'attention_scores':values[i,j,k,x,y],
                  'layer':(i+1), 'head':(k+1),
                  'rank':(y+1),
                  'batch':j}
              dataframe.append(d)
    df = pd.DataFrame(dataframe)
    return df

In [23]:
#combine the previous functions
def highest_tokens(matrix, k, all_tokens, example=None):
  index, values = find_top_attention_unsummed(matrix, k)
  highest_tokens = get_tokens(index, example)
  df = highest_attended_tokens(index, values, highest_tokens, all_tokens, example)
  return df

Thus, for every token, we can get the top k tokens that this token attends to. We can filter by batch, layer, head, rank, position, etc.,. The downsides are that its not very visually appealing despite being organized.

In [24]:
df2 = highest_tokens(converted_mat_importance, 10, all_tokens, 0)
df2

,token,self_position,attended_token,token_position,attention_scores,layer,head,rank,batch
0,<s>,0,<s>,0,0.001840,1,1,1,0
1,<s>,0,-,512,0.000818,1,1,2,0
2,<s>,0,Ġa,1536,0.000718,1,1,3,0
3,<s>,0,Ġimage,1024,0.000706,1,1,4,0
4,<s>,0,Ġcaption,1309,0.000064,1,1,5,0
...,...,...,...,...,...,...,...,...,...
2949115,</s>,2047,.,2006,0.000011,12,12,6,0
2949116,</s>,2047,Ġlearns,1993,0.000011,12,12,7,0
2949117,</s>,2047,.,1792,0.000011,12,12,8,0
2949118,</s>,2047,.,1909,0.000011,12,12,9,0


#### Total attention for each word

Get the sum of the attentions for all the tokens (column-wise). In other words, find out how much every word is attended to.

In [25]:
attention_matrix_importance = converted_mat_importance.sum(axis=3)
print(attention_matrix_importance.shape)

(12, 1, 12, 2048)


#### PDF view

A dataframe is good for picking out information from the example, but it isn't the best being a easy to read visualization. Its easier to see how much each word is attended to in an example if we have the actual example, with the words highlighted based on the magnitude of attention.

We use https://github.com/jiesutd/Text-Attention-Heatmap-Visualization to show how much each token in the example is attended to, up to the max number of tokens we specified earlier.

Get top k attended words for each head, for each example in batch, for each layer

In [26]:
## convert the text/attention list to latex code, which will further generates the text heatmap based on attention weights.
import numpy as np

latex_special_token = ["!@#$%^&*(){}"]

def generate(text_list, attention_list, latex_file, color='red', rescale_value = False):
	assert(len(text_list) == len(attention_list))
	if rescale_value:
		attention_list = rescale(attention_list)
	word_num = len(text_list)
	text_list = clean_word(text_list)
	with open(latex_file,'w') as f:
		f.write(r'''\documentclass[varwidth]{standalone}
\special{papersize=210mm,297mm}
\usepackage{color}
\usepackage{tcolorbox}
\usepackage{CJK}
\usepackage{adjustbox}
\tcbset{width=0.9\textwidth,boxrule=0pt,colback=red,arc=0pt,auto outer arc,left=0pt,right=0pt,boxsep=5pt}
\begin{document}
\begin{CJK*}{UTF8}{gbsn}'''+'\n')
		string = r'''{\setlength{\fboxsep}{0pt}\colorbox{white!0}{\parbox{0.9\textwidth}{'''+"\n"
		for idx in range(word_num):
			string += "\\colorbox{%s!%s}{"%(color, attention_list[idx])+"\\strut " + text_list[idx]+"} "
		string += "\n}}}"
		f.write(string+'\n')
		f.write(r'''\end{CJK*}
\end{document}''')

def rescale(input_list):
	the_array = np.asarray(input_list)
	the_max = np.max(the_array)
	the_min = np.min(the_array)
	rescale = (the_array - the_min)/(the_max-the_min)*100
	return rescale.tolist()


def clean_word(word_list):
	new_word_list = []
	for word in word_list:
		for latex_sensitive in ["\\", "%", "&", "^", "#", "_",  "{", "}"]:
			if latex_sensitive in word:
				word = word.replace(latex_sensitive, '\\'+latex_sensitive)
		new_word_list.append(word)
	return new_word_list

In [27]:
def normalize(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

In [28]:
def find_top_attention(scores_mat, k):
  indices = scores_mat.argsort(axis=3)[:, :, :, :-(k+1):-1]
  vals = np.take_along_axis(scores_mat, indices, axis=3)
  return indices, vals

Change "papers" in the following to news/whatever alternative dataset if you change the dataset used

In [29]:
print(all_tokens)
average_attention = attention_matrix_importance.squeeze().sum(axis=1)
average_attention = average_attention.sum(axis=0)
average_attention = normalize(average_attention)

['<s>', 'Published', 'Ġas', 'Ġa', 'Ġconference', 'Ġpaper', 'Ġin', 'ĠInternational', 'ĠConference', 'Ġof', 'ĠComputer', 'ĠVision', 'Ġ(', 'IC', 'CV', ')', 'Ġ2017', 'Ġ', 'ĠSpeaking', 'Ġthe', 'ĠSame', 'ĠLanguage', ':', 'ĠMatch', 'ing', 'ĠMachine', 'Ġto', 'ĠHuman', 'ĠCapt', 'ions', 'Ġby', 'ĠAd', 'vers', 'arial', 'ĠTraining', 'ĠRak', 'sh', 'ith', 'ĠShe', 'tty', '1', 'Ġ', 'ĠMarcus', 'ĠRoh', 'r', 'bach', '2', ',', '3', 'Ġ', 'Ġar', 'X', 'iv', ':', '17', '03', '.', '10', '476', 'v', '2', 'Ġ[', 'cs', '.', 'CV', ']', 'Ġ6', 'ĠNov', 'Ġ2017', 'Ġ', 'ĠMario', 'ĠFritz', '1', 'Ġ1', 'Ġ', 'ĠLisa', 'ĠAnne', 'ĠHendricks', '2', 'Ġ', 'ĠBer', 'nt', 'ĠS', 'chie', 'le', '1', 'Ġ', 'ĠMax', 'ĠPlan', 'ck', 'ĠInstitute', 'Ġfor', 'ĠIn', 'format', 'ics', ',', 'ĠSa', 'ar', 'land', 'ĠIn', 'format', 'ics', 'ĠCampus', ',', 'ĠSa', 'arb', 'ru', 'Ì', 'Ī', 'ck', 'en', ',', 'ĠGermany', 'Ġ2', 'Ġ3', 'ĠUC', 'ĠBerkeley', 'ĠE', 'EC', 'S', ',', 'ĠCA', ',', 'ĠUnited', 'ĠStates', 'ĠFacebook', 'ĠAI', 'ĠResearch', 'Ġ', 'ĠAbstract', 'ĠWhil

In [30]:
title_all = f"papers_{test_val[0]}.tex"
generate(all_tokens, (average_attention*100), title_all, 'red')

Adding up all the heads isn't probably the best way of finding out the important tokens of the example because the last layer is what drives the prediction. This part focuses on only the last layer's attention for visualization.

In [31]:
print(attention_matrix_importance[11].squeeze().shape)
average_attention_final_layer = attention_matrix_importance[11].squeeze().sum(axis=0)
average_attention_final_layer = normalize(average_attention_final_layer)

(12, 2048)


In [32]:
title_last_layer = f"papers_{test_val[0]}_layer_12_only.tex"
generate(all_tokens, (average_attention_final_layer*100), title_last_layer, 'red')

#### Highest attended tokens

We want the position (index) of the token, the attention value, and the actual token itself.

In [33]:
def get_tokens2(index_matrix, example=None):
  highest_tokens = []
  #layer
  for i in range(index_matrix.shape[0]):
    row_tokens = []
    #batch
    for j in range(index_matrix.shape[1]):
      batch_tokens = []

      if example is not None and j != example:
        continue

      #head
      for k in range(index_matrix.shape[2]):
        tokens = [all_tokens[idx] for idx in index_matrix[i][j][k]]
        batch_tokens.append(tokens)
      row_tokens.append(batch_tokens)
    highest_tokens.append(row_tokens)
  return np.array(highest_tokens)

In [34]:
def highest_attended_dataframe(index, values, tokens, example=None):
    dataframe=[]
    #layer
    for i in range(index.shape[0]):
      #batch
      for j in range(index.shape[1]):
        if example is not None and j != example:
          continue
        #head
        for k in range(index.shape[2]):
          #token
          for x in range(index.shape[3]):
            d = {"token":tokens[i,j,k,x], 'position':index[i,j,k,x], 
                'attention_scores':values[i,j,k,x],
                 'layer':(i+1), 'head':(k+1),
                 'rank':(x+1),
                 'batch':j}
            dataframe.append(d)
    df = pd.DataFrame(dataframe)
    return df

In [35]:
def highest_attentions_summed(matrix, k, all_tokens, example=None):
  index, values = find_top_attention(matrix, k)
  highest_tokens = get_tokens2(index, example)
  df_highest = highest_attended_dataframe(index, values, highest_tokens, example)
  return df_highest

In [36]:
df_highest = highest_attentions_summed(attention_matrix_importance, 10, all_tokens, 0)
df_highest

,token,position,attention_scores,layer,head,rank,batch
0,<s>,0,0.828359,1,1,1,0
1,-,512,0.197836,1,1,2,0
2,Ġa,1536,0.183238,1,1,3,0
3,Ġimage,1024,0.176865,1,1,4,0
4,ĠAt,1620,0.099787,1,1,5,0
...,...,...,...,...,...,...,...
1435,.,598,0.007779,12,12,6,0
1436,.,572,0.006871,12,12,7,0
1437,.,557,0.006470,12,12,8,0
1438,.,533,0.005772,12,12,9,0


Get the attention of a token at a position for each layer and head as well as the rank of the token within that head. Take one example at a time as each example has different tokens. 

Pros: can isolate for layers and/or heads. Cons: not much context for the attention scores

In [37]:
def position_attention(agg_matrix, position, example=None):
  dataframe=[]
  if example is not None:
    new_mat = agg_matrix[:, example, :]
    new_mat = new_mat.squeeze()
    #layer
    for i in range(new_mat.shape[0]):
      #head
      for j in range(new_mat.shape[1]):
        temp = new_mat[i,j].argsort()[::-1]
        temp = np.where(temp==position)[0].squeeze() + 1
        d = {"token":all_tokens[position], 'position':position, 
            'attention_scores':new_mat[i,j,position], 'layer':(i+1), 'head':(j+1),
            'rank':temp, 'batch':example}
        dataframe.append(d)
  else:
    new_mat = agg_matrix
    #layer
    for i in range(new_mat.shape[0]):
      #batch
      for j in range(new_mat.shape[1]):
        #head
        for k in range(new_mat.shape[2]):
          temp = new_mat[i,j,k].argsort()[::-1]
          temp = np.where(temp==position)[0].squeeze() + 1
          d = {"token":all_tokens[position], 'position':position, 
              'attention_scores':new_mat[i,j,k,position], 'layer':(i+1), 'head':(k+1),
              'rank':temp, 'batch':j}
          dataframe.append(d)
  df = pd.DataFrame(dataframe)
  return df

In [38]:
position_df = position_attention(attention_matrix_importance, 0)
position_df[position_df["head"]==1]

,token,position,attention_scores,layer,head,rank,batch
0,<s>,0,0.828359,1,1,1,0
12,<s>,0,2.288568,2,1,1,0
24,<s>,0,2.218065,3,1,1,0
36,<s>,0,30.302418,4,1,1,0
48,<s>,0,1.400105,5,1,1,0
60,<s>,0,1.034879,6,1,1,0
72,<s>,0,0.147366,7,1,122,0
84,<s>,0,0.106004,8,1,1,0
96,<s>,0,0.000766,9,1,38,0
108,<s>,0,0.000000,10,1,2048,0


If really needed, can just have the full matrix of the position, ranks, attention scores, layers, and heads of each token per example. 

Tokens are all the tokens in the example. Position is the location of the token with zero-based indexing. Attention scores are the aggregate, scaled attention scores.

Layer goes from 1 to 12.
Head goes from 1 to 12.

Rank is the attention score rank with respect to layer and head. Goes from 1 to number of tokens in the example

Pros: can search up whatever is needed. Has access to all the information and can be extracted for comparisons Cons: have to know what you want and manually look it up

In [39]:
def full_matrix(agg_matrix, example=None):
  dataframe=[]

  if example is not None:
    new_mat = agg_matrix[:, example]
    new_mat = new_mat.squeeze()
    print(new_mat.shape)

    #layer
    for i in range(new_mat.shape[0]):
      #head
      for j in range(new_mat.shape[1]):
        temp = new_mat[i,j].argsort()[::-1]      
        #token
        for k in range(new_mat.shape[2]):
          temp2 = np.where(temp==k)[0].squeeze() + 1
          d = {"token":all_tokens[k], 'position':k, 
              'attention_scores':new_mat[i,j,k], 'layer':(i+1), 'head':(j+1),
              'rank':temp2}
          dataframe.append(d)
  else:
    new_mat = agg_matrix
    print(new_mat.shape)
    
    #layer
    for i in range(new_mat.shape[0]):
      #batch
      for j in range(new_mat.shape[1]):
        #head
        for k in range(new_mat.shape[2]):
          temp = new_mat[i,j,k].argsort()[::-1]      
          #token
          for x in range(new_mat.shape[3]):
            temp2 = np.where(temp==x)[0].squeeze() + 1
            d = {"token":all_tokens[x], 'position':x, 
                'attention_scores':new_mat[i,j,k,x], 'layer':(i+1), 'head':(k+1),
                'rank':temp2, 'batch':j}
            dataframe.append(d)
  df = pd.DataFrame(dataframe)
  return df

In [40]:
full_mat = full_matrix(attention_matrix_importance)

(12, 1, 12, 2048)


In [41]:
full_mat[(full_mat['head']==11) & (full_mat['layer']==11)]

,token,position,attention_scores,layer,head,rank,batch
266240,<s>,0,0.003133,11,11,653,0
266241,Published,1,0.002397,11,11,721,0
266242,Ġas,2,0.000142,11,11,1743,0
266243,Ġa,3,0.000198,11,11,1622,0
266244,Ġconference,4,0.002712,11,11,687,0
...,...,...,...,...,...,...,...
268283,Ġgenerate,2043,0.002030,11,11,772,0
268284,Ġreal,2044,0.002215,11,11,744,0
268285,Ġimages,2045,0.010886,11,11,372,0
268286,",",2046,0.000265,11,11,1507,0
